In [25]:
# EfficientNet-B5 문서 이미지 분류 모델
# Train 데이터: 1570장 (깨끗한 이미지)
# Test 데이터: 3140장 (회전, 플립, 노이즈가 심한 이미지)
# 클래스: 17개 (불균형 데이터)

import os
import random
import numpy as np
import pandas as pd
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import StratifiedKFold
import timm
import albumentations as A
from albumentations.pytorch import ToTensorV2
import wandb
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

In [56]:
# ================================
# 8. Learning Rate Scheduler (Warmup + Cosine Annealing)
# ================================
class WarmupCosineScheduler:
    def __init__(self, optimizer, warmup_epochs, total_epochs, max_lr, min_lr):
        self.optimizer = optimizer
        self.warmup_epochs = warmup_epochs
        self.total_epochs = total_epochs
        self.max_lr = max_lr
        self.min_lr = min_lr
        
    def step(self, epoch):
        if epoch < self.warmup_epochs:
            # Warmup: 선형 증가
            lr = self.max_lr * (epoch + 1) / self.warmup_epochs
        else:
            # Cosine Annealing: 코사인 함수로 감소
            progress = (epoch - self.warmup_epochs) / (self.total_epochs - self.warmup_epochs)
            lr = self.min_lr + (self.max_lr - self.min_lr) * 0.5 * (1 + np.cos(np.pi * progress))
        
        for param_group in self.optimizer.param_groups:
            param_group['lr'] = lr
        
        return lr

In [57]:
# ================================
# 9. 모델 생성
# ================================
def create_model(model_name, num_classes, pretrained=True):
    model = timm.create_model(
        model_name,
        pretrained=pretrained,
        num_classes=num_classes
    )
    return model

In [58]:
# ================================
# 10. 학습 함수
# ================================
def train_one_epoch(model, dataloader, criterion, optimizer, device, epoch):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    pbar = tqdm(dataloader, desc=f'Epoch {epoch+1} - Training')
    for images, labels in pbar:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
        
        pbar.set_postfix({'loss': loss.item(), 'acc': 100 * correct / total})
    
    epoch_loss = running_loss / total
    epoch_acc = 100 * correct / total
    
    return epoch_loss, epoch_acc


In [59]:
# ================================
# 11. 검증 함수
# ================================
def validate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in tqdm(dataloader, desc='Validating'):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    
    epoch_loss = running_loss / total
    epoch_acc = 100 * correct / total
    
    return epoch_loss, epoch_acc


In [60]:
# ================================
# 12. K-Fold Cross Validation 학습
# ================================
def train_with_kfold():
    # 데이터 로드
    train_df = pd.read_csv(config.train_csv)
    meta_df = pd.read_csv(config.meta_csv)
    
    # Stratified K-Fold 설정 (클래스 불균형 고려)
    skf = StratifiedKFold(n_splits=config.n_folds, shuffle=True, random_state=42)
    
    fold_results = []
    
    for fold, (train_idx, valid_idx) in enumerate(skf.split(train_df, train_df['target'])):
        print(f'\n{"="*50}')
        print(f'Fold {fold+1}/{config.n_folds}')
        print(f'{"="*50}')
        
        # Fold별 데이터 분리
        train_fold_df = train_df.iloc[train_idx].reset_index(drop=True)
        valid_fold_df = train_df.iloc[valid_idx].reset_index(drop=True)
        
        # 데이터셋 및 데이터로더
        train_dataset = DocumentDataset(
            train_fold_df, 
            config.train_img_dir, 
            transform=get_train_transform(config.img_size)
        )
        valid_dataset = DocumentDataset(
            valid_fold_df, 
            config.train_img_dir, 
            transform=get_valid_transform(config.img_size)
        )
        
        train_loader = DataLoader(
            train_dataset, 
            batch_size=config.batch_size, 
            shuffle=True, 
            num_workers=config.num_workers,
            pin_memory=True
        )
        valid_loader = DataLoader(
            valid_dataset, 
            batch_size=config.batch_size, 
            shuffle=False, 
            num_workers=config.num_workers,
            pin_memory=True
        )
        
        # 모델 생성
        model = create_model(config.model_name, config.num_classes, pretrained=True)
        model = model.to(config.device)
        
        # Loss 및 Optimizer
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.AdamW(model.parameters(), lr=config.max_lr, weight_decay=1e-4)
        
        # Learning Rate Scheduler
        scheduler = WarmupCosineScheduler(
            optimizer, 
            warmup_epochs=config.warmup_epochs,
            total_epochs=config.epochs,
            max_lr=config.max_lr,
            min_lr=config.min_lr
        )
        
        # 학습
        best_val_acc = 0.0
        
        for epoch in range(config.epochs):
            # Learning rate 업데이트
            current_lr = scheduler.step(epoch)
            
            # 학습
            train_loss, train_acc = train_one_epoch(
                model, train_loader, criterion, optimizer, config.device, epoch
            )
            
            # 검증
            val_loss, val_acc = validate(model, valid_loader, criterion, config.device)
            
            print(f'Epoch {epoch+1}/{config.epochs}')
            print(f'LR: {current_lr:.6f}')
            print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%')
            print(f'Valid Loss: {val_loss:.4f}, Valid Acc: {val_acc:.2f}%')
            
            # WandB 로깅
            if config.use_wandb:
                wandb.log({
                    f'fold_{fold+1}/train_loss': train_loss,
                    f'fold_{fold+1}/train_acc': train_acc,
                    f'fold_{fold+1}/val_loss': val_loss,
                    f'fold_{fold+1}/val_acc': val_acc,
                    f'fold_{fold+1}/learning_rate': current_lr,
                    'epoch': epoch + 1,
                })
            
            # Best 모델 저장
            if val_acc > best_val_acc:
                best_val_acc = val_acc
                torch.save(
                    model.state_dict(), 
                    f'best_model_fold_{fold+1}.pth'
                )
                print(f'✓ Best model saved! (Val Acc: {val_acc:.2f}%)')
        
        fold_results.append(best_val_acc)
        print(f'\nFold {fold+1} Best Validation Accuracy: {best_val_acc:.2f}%')
    
    # 전체 Fold 결과
    print(f'\n{"="*50}')
    print('Cross-Validation Results:')
    print(f'{"="*50}')
    for i, acc in enumerate(fold_results):
        print(f'Fold {i+1}: {acc:.2f}%')
    print(f'Mean CV Accuracy: {np.mean(fold_results):.2f}% ± {np.std(fold_results):.2f}%')
    
    if config.use_wandb:
        wandb.log({
            'cv_mean_accuracy': np.mean(fold_results),
            'cv_std_accuracy': np.std(fold_results),
        })

In [61]:
# ================================
# 13. 테스트 예측 (TTA 적용)
# ================================
def predict_with_tta():
    """TTA를 적용한 테스트 예측"""
    submission_df = pd.read_csv(config.submission_csv)
    
    # TTA 변형들
    tta_transforms = get_tta_transforms(config.img_size)
    
    # 모든 Fold 모델 로드
    models = []
    for fold in range(config.n_folds):
        model = create_model(config.model_name, config.num_classes, pretrained=False)
        model.load_state_dict(torch.load(f'best_model_fold_{fold+1}.pth'))
        model = model.to(config.device)
        model.eval()
        models.append(model)
    
    all_predictions = []
    
    # 각 테스트 이미지에 대해 예측
    for img_name in tqdm(submission_df['ID'], desc='Predicting with TTA'):
        img_path = os.path.join(config.test_img_dir, img_name)
        image = Image.open(img_path).convert('RGB')
        image = np.array(image)
        
        tta_preds = []
        
        # 각 TTA 변형에 대해
        for transform in tta_transforms:
            augmented = transform(image=image)
            img_tensor = augmented['image'].unsqueeze(0).to(config.device)
            
            # 각 Fold 모델에 대해
            fold_preds = []
            with torch.no_grad():
                for model in models:
                    outputs = model(img_tensor)
                    probs = torch.softmax(outputs, dim=1)
                    fold_preds.append(probs.cpu().numpy())
            
            # Fold 평균
            fold_avg = np.mean(fold_preds, axis=0)
            tta_preds.append(fold_avg)
        
        # TTA 평균
        final_pred = np.mean(tta_preds, axis=0)
        predicted_class = np.argmax(final_pred)
        all_predictions.append(predicted_class)
    
    # 결과 저장
    submission_df['target'] = all_predictions
    submission_df.to_csv('submission.csv', index=False)
    print('Submission file saved!')
    
    if config.use_wandb:
        wandb.save('submission.csv')

In [40]:
# ⬇️ 1. [사용자 입력 필요] 각 Fold의 검증(Validation) F1 스코어 (또는 정확도)
# 예시: 5-Fold 였다면 5개의 점수를 리스트로 제공 (순서 중요!)
all_fold_scores = [
    0.9682,  # Fold 1의 검증 F1 스코어
    0.9490,  # Fold 2의 검증 F1 스코어
    0.9495,  # Fold 3의 검증 F1 스코어
    0.9522,  # Fold 4의 검증 F1 스코어
    0.9554   # Fold 5의 검증 F1 스코어
    # (config.n_folds 개수와 일치해야 함)
]

# ================================
# 13. 테스트 예측 (TTA 적용) - 가중 평균 버전
# ================================
def predict_with_tta():
    """TTA를 적용한 테스트 예측 (Validation 스코어 기반 가중 평균)"""
    submission_df = pd.read_csv(config.submission_csv)
    
    # TTA 변형들
    tta_transforms = get_tta_transforms(config.img_size)
    
    # 존재하는 Fold 모델 로드 및 해당 스코어 매칭
    models = []
    loaded_scores = []  # ⬇️ 2. [수정] 로드된 모델에 해당하는 스코어를 저장할 리스트
    
    for fold in range(config.n_folds):
        model_path = f'best_model_fold_{fold+1}.pth'
        
        # 파일이 존재하는지 확인
        if not os.path.exists(model_path):
            print(f"⚠️  {model_path} not found. Skipping this fold.")
            continue
            
        model = create_model(config.model_name, config.num_classes, pretrained=False)
        model.load_state_dict(torch.load(model_path))
        model = model.to(config.device)
        model.eval()
        models.append(model)
        
        # ⬇️ 3. [수정] 모델이 로드될 때, 해당 Fold의 스코어도 함께 추가
        loaded_scores.append(all_fold_scores[fold])
        print(f"✓ Loaded {model_path} (Score: {all_fold_scores[fold]})")
    
    if len(models) == 0:
        print("❌ Error: No model files found!")
        return
        
    # ⬇️ 4. [수정] 로드된 스코어를 기반으로 가중치(Weights) 계산
    #    (전체 합이 1이 되도록 정규화)
    weights = np.array(loaded_scores)
    weights = weights / np.sum(weights)
    
    print(f"\n{'='*50}")
    print(f"Using {len(models)} fold model(s) for WEIGHTED prediction")
    print(f"Scores: {loaded_scores}")
    print(f"Weights: {[round(w, 4) for w in weights]}") # 계산된 가중치 출력
    print(f"{'='*50}\n")
    
    all_predictions = []
    
    # 각 테스트 이미지에 대해 예측
    for img_name in tqdm(submission_df['ID'], desc='Predicting with TTA'):
        img_path = os.path.join(config.test_img_dir, img_name)
        image = Image.open(img_path).convert('RGB')
        image = np.array(image)
        
        tta_preds = []
        
        # 각 TTA 변형에 대해
        for transform in tta_transforms:
            augmented = transform(image=image)
            img_tensor = augmented['image'].unsqueeze(0).to(config.device)
            
            # 각 Fold 모델에 대해
            fold_preds = []
            with torch.no_grad():
                for model in models:
                    outputs = model(img_tensor)
                    probs = torch.softmax(outputs, dim=1)
                    fold_preds.append(probs.cpu().numpy())
            
            # ⬇️ 5. [수정] Fold 가중 평균 (np.mean -> np.average)
            # np.average 함수에 weights 파라미터를 전달하여 가중 평균 계산
            fold_avg = np.average(fold_preds, axis=0, weights=weights) 
            tta_preds.append(fold_avg)
        
        # TTA 평균 (이 부분은 동일)
        final_pred = np.mean(tta_preds, axis=0)
        predicted_class = np.argmax(final_pred)
        all_predictions.append(predicted_class)
    
    # 결과 저장
    submission_df['target'] = all_predictions
    submission_df.to_csv('submission_weighted.csv', index=False) # 파일 이름 변경
    print('\n✓ Submission file (weighted) saved!')
    
    if config.use_wandb:
        wandb.save('submission_weighted.csv')
    
    # 예측 분포 출력
    print(f"\n{'='*50}")
    print("Predicted class distribution (Weighted):")
    print(f"{'='*50}")
    unique, counts = np.unique(all_predictions, return_counts=True)
    for cls, cnt in zip(unique, counts):
        print(f"  Class {cls}: {cnt} images ({cnt/len(all_predictions)*100:.1f}%)")